# Basic EDA


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
path = '/content/gdrive/My Drive/Flipr 6//'

In [3]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [4]:
df = pd.read_excel(path+'Test-dataset.xlsx', sheet_name="Foreign_Visitors_TS")

In [5]:
df  =  df.iloc[1:]
df.columns  =  ['City', 'April', 'May', 'June', 'July', 'August']
from sklearn.utils import shuffle
df  =  shuffle(df)

In [6]:
df.head()

,City,April,May,June,July,August
94,Sarangpur,NaN,294964,358178,379237,421365
137,Sitarganj,61420,74125,90018,95302,105882
279,Lohardaga,97323,117458,142635,151015,167785
100,Patratu,97323,117458,142635,151015,167785
454,Bagha Kusmar,535775,646624,785194,831371,923737


# Relation between the columns?

#### Using PyCaret for quick prediction.

In [7]:
del df['City']
display(df.shape)
df = df.dropna()
display(df.shape)

(501, 5)

(407, 5)

In [8]:
try:
    from pycaret.regression import *
except:
    !pip install pycaret==2.0
    from pycaret.regression import *

In [9]:
# Train Test Split.
train_, test_  =  df.iloc[:int(0.8*len(df))], df.iloc[int(0.8*len(df)):]
df.shape, train_.shape, test_.shape

((407, 5), (325, 5), (82, 5))

In [10]:
train_.head()

,April,May,June,July,August
137,61420,74125,90018,95302,105882
279,97323,117458,142635,151015,167785
100,97323,117458,142635,151015,167785
454,535775,646624,785194,831371,923737
231,2717138,3279303,3982019,4216244,4684707


In [11]:
reg  =  setup(train_, 
              target = 'August')

 
Setup Succesfully Completed.


,Description,Value
0,session_id,6663
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(325, 5)"
4,Missing Values,False
5,Numeric Features,4
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [12]:
best_model = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Linear Regression,0.0717,0.0127,0.1126,1.0000,0.0000,0.0000,0.0085
1,Orthogonal Matching Pursuit,0.1780,0.0525,0.2276,1.0000,0.0000,0.0000,0.0042
2,Gradient Boosting Regressor,125.4499,378872.1687,428.1906,1.0000,0.0468,0.0149,0.0309
3,Extra Trees Regressor,12.3926,19176.8576,79.2276,1.0000,0.0370,0.0109,0.1430
4,Ridge Regression,0.0723,0.0130,0.1136,1.0000,0.0000,0.0000,0.0042
5,Elastic Net,0.4475,0.2519,0.5011,1.0000,0.0000,0.0000,0.0055
6,Least Angle Regression,0.0733,0.0129,0.1131,1.0000,0.0000,0.0000,0.0051
7,Lasso Least Angle Regression,12.1251,185.8799,13.5735,1.0000,0.0008,0.0002,0.0056
8,Bayesian Ridge,1857.3892,23050595.7572,2152.9463,1.0000,0.0513,0.0255,0.0190
9,Lasso Regression,0.4475,0.2519,0.5011,1.0000,0.0000,0.0000,0.0052


Since even the simplest models show almost perfect score there is a strong relation, digging a little deeper we find the following - 

In [13]:
# df['May']     =  df['April']*1.2068969800369 - 2.612513869 + c
# df['June']    =  df['May']  *1.2142848513404 + 7.683109226 + c
# df['July']    =  df['June'] *1.0588299671675 - 12.5999801  + c
# df['August']  =  df['July'] *1.1111038027494 - 6.672843276 + c

# where c is a very small constant.


# BELOW IS THE VALIDATION - MAPE is almost 0.
print(  mean_absolute_percentage_error(df['April']*1.2068969800369 -2.612513869, df['May']), \
        mean_absolute_percentage_error(df['May']  *1.2142848513404 + 7.683109226, df['June']), \
        mean_absolute_percentage_error(df['June'] *1.0588299671675 - 12.5999801,  df['July']), \
        mean_absolute_percentage_error(df['July'] *1.1111038027494 - 6.672843276, df['August'])  )


0.0016124273940893684 0.00101758632382994 0.0022663719934442462 0.002861744639518736


### As constants are negligible compared to the absolute values, let's ignore and focus only on the '% change'
### From April to May:     +20.7%
### From May to June:      +21.4%
### From June to July:     +05.9%
### From July to August:   +11.1%

### Essentially, we have only 5 data points in the dataset, so we calculate 
### '% Change From August to September' = Average of Median + Mean

In [15]:
percentage_change_list = np.array([0.2068969800369, 0.2142848513404, 0.0588299671675, 0.1111038027494])
percentage_change = (np.mean(percentage_change_list)  +  np.median(percentage_change_list))  /  2
percentage_change

0.15338964585835002

### So, Sepetember will have an increase in cases from August by 15.3%. <br> [And, we use this to predict in 01.ipynb]